<img style="float: right;" src="../htwlogo.jpg">

# Gradients with pytorch

**Author**: Dive into Deep Learning, adapted by _Erik Rodner_<br>
**Lecture**: Computer Vision and Machine Learning I

In the following exercise, we will look at automatic differentiation.

https://d2l.ai/chapter_preliminaries/autograd.html

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
x.grad  # The default value is None

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [6]:
z = torch.ones(4, requires_grad=True)

In [11]:
y = torch.dot(x, z)

In [12]:
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [13]:
y.backward()

In [14]:
x.grad

tensor([1., 1., 1., 1.])

In [15]:
z.grad

tensor([0., 1., 2., 3.])